<a href="https://colab.research.google.com/github/thomson008/pdiot-cw3/blob/main/Data_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [68]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from scipy import stats
import keras
from keras.layers import Conv1D, Dropout, Dense, MaxPooling1D, Flatten
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [148]:
path = '/content/drive/My Drive/Colab Notebooks/pdiot-cw3/data'

file_list = []
sensor_position = 'Chest_Right'

for root, dirs, files in os.walk(path):
    for file in files:
        if ('csv' in file and sensor_position in file):
            file_list.append(os.path.join(root, file))
            
dataframes = []

for file in file_list:
    with open(file) as f:
        info = [next(f).rstrip().split(': ')[1] for x in range(5)]
        user_id = info[4]
        activity = info[2]
        
    file_data = pd.read_csv(file, header=5)
    if (file_data.size):
        dataframes.append(file_data)
        file_data.insert(0, 'user_id', user_id)
        file_data.insert(1, 'activity', activity)
    

sensor_data = pd.concat(dataframes, ignore_index=True).drop('seq', axis=1)
sensor_data.to_csv('sensor.csv')

In [200]:
subjects = sensor_data.user_id.unique()
train_subjects, test_subjects = subjects[: int(0.98*len(subjects))], subjects[int(0.98*len(subjects)):]

df_train, df_test = sensor_data[sensor_data.user_id != 's1758009'], sensor_data[sensor_data.user_id == 's1758009']

scale_columns = ['accel_x', 'accel_y', 'accel_z']

scaler = RobustScaler()

scaler = scaler.fit(df_train[scale_columns])

df_train.loc[:, scale_columns] = scaler.transform(
  df_train.loc[:, scale_columns].to_numpy()
)

df_test.loc[:, scale_columns] = scaler.transform(
  df_test.loc[:, scale_columns].to_numpy()
)

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1736: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [205]:
def create_dataset(X, y, time_steps=1, step=1):
    Xs, ys = [], []
    for i in range(0, len(X) - time_steps, step):
        v = X.iloc[i:(i + time_steps)].values
        labels = y.iloc[i: i + time_steps]
        Xs.append(v)
        ys.append(stats.mode(labels)[0][0])
    return np.array(Xs), np.array(ys).reshape(-1, 1)

TIME_STEPS = 36
STEP = 10

X_train, y_train = create_dataset(
    df_train[scale_columns],
    df_train.activity,
    TIME_STEPS,
    STEP
)

X_test, y_test = create_dataset(
    df_test[scale_columns],
    df_test.activity,
    TIME_STEPS,
    STEP
)

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(y_train)

y_train = enc.transform(y_train)
y_test = enc.transform(y_test)

In [207]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 10, 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    model = keras.Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(n_timesteps,n_features)))
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

evaluate_model(X_train, y_train, X_test, y_test)

Epoch 1/10
212/212 [==============================] - 2s 8ms/step - loss: 1.2784 - accuracy: 0.5590
Epoch 2/10
212/212 [==============================] - 2s 8ms/step - loss: 0.7764 - accuracy: 0.7102
Epoch 3/10
212/212 [==============================] - 2s 8ms/step - loss: 0.6230 - accuracy: 0.7768
Epoch 4/10
212/212 [==============================] - 2s 8ms/step - loss: 0.5232 - accuracy: 0.8102
Epoch 5/10
212/212 [==============================] - 2s 10ms/step - loss: 0.4707 - accuracy: 0.8290
Epoch 6/10
212/212 [==============================] - 2s 10ms/step - loss: 0.4156 - accuracy: 0.8483
Epoch 7/10
212/212 [==============================] - 2s 9ms/step - loss: 0.3903 - accuracy: 0.8575
Epoch 8/10
212/212 [==============================] - 2s 8ms/step - loss: 0.3611 - accuracy: 0.8675
Epoch 9/10
212/212 [==============================] - 2s 8ms/step - loss: 0.3254 - accuracy: 0.8836
Epoch 10/10
212/212 [==============================] - 2s 8ms/step - loss: 0.3130 - accuracy: 0.88

0.6930320262908936

In [ ]:
|